In [3]:
import psycopg2 as psycopg2
from string import Template
import pylab

conn = psycopg2.connect(
    host="localhost",
    database="drugbank",
    user="vlad",
    password="vlad",
    port=5433)

In [4]:
### Process Data
drugCommunityDistribution={}
drugCommunities={}
# 181
# 137
for i in range(0,134):
    communityDistribution = {
    "V": set(), 
    "M": set(), 
    "D": set(), 
    "B": set(), 
    "S": set(), 
    "N": set(), 
    "L": set(), 
    "G": set(), 
    "J": set(), 
    "P": set(), 
    "A": set(), 
    "R": set(), 
    "C": set(), 
    "H": set() 
    } 

#     i = 56
    cursor = conn.cursor()

    t = """select drugs.name, drugs_resolution.modularity_class, drugs.atc_code as first_letter 
            FROM public.all_drugs as drugs
            LEFT JOIN public.drugs_resolution 
            ON drugs.name = drugs_resolution.name 
            WHERE CARDINALITY(atc_code) != 0 AND cardinality(drugs.gene_target_ids)!= 0 
                AND drugs_resolution.drugbank_version = '4.1'
                AND drugs_resolution.resolution = 1
                AND drugs_resolution.modularity_class = %s
            """


#     t = """select drugs.name, temp_drug_modularity.modularity_class, drugs.atc_code as first_letter FROM public."drugs_4.1" as drugs
#     LEFT JOIN public.temp_drug_modularity as temp_drug_modularity
#     ON drugs.name = temp_drug_modularity.id 
#     WHERE CARDINALITY(atc_code) != 0 AND cardinality(drugs.gene_target_ids)!= 0 
#         AND temp_drug_modularity.modularity_class = """ + str(i) + """
#     """
    
    #     t.format(modularityClass=modularityClass)
    cursor.execute(t,(i,))

    # display the PostgreSQL database server version
    drugs = cursor.fetchall()
    drugCommunities[i]=drugs
    # close the communication with the PostgreSQL
    cursor.close()

    for j in range(0,len(drugs)):
        for k in range(0, len(drugs[j][2])):
            atcCode = drugs[j][2][k]
            firstAtcLetter = atcCode[0:1]
            communityDistribution[firstAtcLetter].add(drugs[j][0])
            
    drugCommunityDistribution[i] = communityDistribution

        
    


In [6]:
excludedDrugsList = list()

for i in range(0,134):
    modularity = i

    maxCount = 0
    maxKey = 'V'
    for key in drugCommunityDistribution[modularity]:
    #     print(key , len(drugCommunityDistribution[modularity][key]))
        if maxCount < len(drugCommunityDistribution[modularity][key]):
            maxCount = len(drugCommunityDistribution[modularity][key])
            maxKey = key

    # print(maxCount)
#     print(maxKey)

    ### Get all the drugs that are not in the exclusion atc group

    def myFunc(x):
      for val in drugCommunityDistribution[modularity][maxKey]:
          if x[0] == val:
            return False

      return True 

    excludedDrugs = list(filter(myFunc, drugCommunities[modularity]))

    # for x in excludedDrugs:
    #   print(x)
    for key in excludedDrugs:
        key = list(key)
        key.append(maxKey)
        key = tuple(key)
        excludedDrugsList.append(key)
    #     print(key)

    # type()


print(((excludedDrugsList)))


[('Dipyridamole', Decimal('0'), ['B01AC07'], 'C'), ('Dipyridamole', Decimal('0'), ['B01AC07'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Dipyridamole', Decimal('0'), ['B01AC07'], 'C'), ('Dipyridamole', Decimal('0'), ['B01AC07'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Dipyridamole', Decimal('0'), ['B01AC07'], 'C'), ('Dipyridamole', Decimal('0'), ['B01AC07'], 'C'), ('Dipyridamole', Decimal('0'), ['B01AC07'], 'C'), ('Vardenafil', Decimal('0'), ['G04BE09'], 'C'), ('Vardenafil', Decimal('0

In [7]:
repurposedDrugs = list()
for i in range (0, len(excludedDrugsList)):
    element = excludedDrugsList[i]

#     print(element)
#     print(element[0])

    query = """SELECT atc_code FROM public.drugs WHERE name = '""" + element[0] + """';"""
#     print(query)

    cursor = conn.cursor()
    cursor.execute(query)

    # display the PostgreSQL database server version
    updated_drug = cursor.fetchall()
    if len(updated_drug) > 0:
        for i in range(0,len(updated_drug[0][0])):
            atcCode = updated_drug[0][0][i]
            firstAtcLetter = atcCode[0:1]
#             print(firstAtcLetter)
            if firstAtcLetter == element[3]:
                repurposedDrugs.append(element)
                break

print(repurposedDrugs)

[]


In [8]:
print(len(excludedDrugsList))

8814


In [9]:
print((repurposedDrugs))

[]
